In [4]:
pip install pickle5

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build pickle5
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Building wheel for pickle5 (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build\lib.win-amd64-cpython-311\pickle5
      copying pickle5\pickle.py -> build\lib.win-amd64-cpython-311\pickle5
      copying pickle5\pickletools.py -> build\lib.win-amd64-cpython-311\pickle5
      copying pickle5\__init__.py -> build\lib.win-amd64-cpython-311\pickle5
      creating build\lib.win-amd64-cpython-311\pickle5\test
      copying pickle5\test\pickletester.py -> build\lib.win-amd64-cpython-311\pickle5\test
      copying pickle5\test\test_pickle.py -> build\lib.win-amd64-cpython-311\pickle5\test
      copying pickle5\test\test_picklebuffer.py -> build\lib.win-amd64-cpython-311\pickle5\test
      copying pickle5\test\__init__.py -> build\lib.win-amd64-cpython-311\pickle5\test
      running build_ext
      building 'pickle5._pickle' exte

In [1]:
import openai
import os
from dotenv import load_dotenv
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import OpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document as LDocument
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
import pymysql
import locale
import gradio as gr
from langchain.text_splitter import RecursiveCharacterTextSplitter

# .env 파일에서 환경 변수 로드
load_dotenv()

# OpenAI API 설정
openai.api_key = os.getenv("OPENAI_API_KEY")

import pickle
# 피클 파일을 저장하는 함수
def save_to_pickle(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)
# 피클 파일을 로드하는 함수
def load_from_pickle(filename):
    if os.path.exists(filename):
        with open(filename, 'rb') as f:
            return pickle.load(f)
    else:
        return None

# MySQL 데이터베이스 연결 함수 (예시)
def connect_to_db():
    return pymysql.connect(
        host='database-1.cxcqeqcc6xxo.ap-northeast-2.rds.amazonaws.com',
        port=3306,
        user='admin',
        passwd='sesaclangchain',
        db='sesaclangchain',
        charset="utf8mb4"
    )

# 온라인 강의와 오프라인 강의 정보를 가져오는 함수
def fetch_courses():
    connection = connect_to_db()
    cursor = connection.cursor(pymysql.cursors.DictCursor)

    # 온라인 강의 정보 가져오기
    online_query = "SELECT * FROM online_course_tbl"
    cursor.execute(online_query)
    online_courses = cursor.fetchall()

    # 오프라인 강의 정보 가져오기
    offline_query = "SELECT * FROM offline_course_tbl"
    cursor.execute(offline_query)
    offline_courses = cursor.fetchall()

    # 공지 
    notice_query = "SELECT * FROM notice_tbl"
    cursor.execute(notice_query)
    notice_info = cursor.fetchall()

    # qna 
    qna_query = 'SELECT * FROM qna_tbl'
    cursor.execute(qna_query)
    qna_info = cursor.fetchall()

    # 리뷰
    review_query = 'select * from review'
    cursor.execute(review_query)
    review_info = cursor.fetchall()

    # 강의 이미지
    course_image_query = 'select * from class_img2txt'
    cursor.execute(course_image_query)
    course_image_info = cursor.fetchall()

    connection.close()

    return online_courses, offline_courses, notice_info, qna_info, review_info,course_image_info

# 강의 정보를 문서 형식으로 변환
# 텍스트를 일정한 길이로 청크 분할하는 함수
def chunk_text(text, chunk_size=500, chunk_overlap=50):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    return text_splitter.split_text(text) if len(text) > chunk_size else [text]  # 너무 짧으면 그대로 반환

# 데이터를 청크로 나누어 문서 리스트로 변환하는 함수
def convert_to_documents(online_courses, offline_courses, notice_info, qna_info, review_info, course_image_info):
    documents = []

    # 데이터를 반복해서 처리하는 공통 함수
    def process_and_chunk(data_list, format_func):
        for item in data_list:
            doc_text = format_func(item)
            chunks = chunk_text(doc_text, chunk_size=500)
            for chunk in chunks:
                documents.append(LDocument(page_content=chunk))

    # 각 데이터에 맞는 텍스트 포맷팅
    process_and_chunk(online_courses, lambda course: (
        f"강의 제목: {course['course_title']}\n"
        f"강의 분류: {course['course_category']}\n"
        f"강의 난도: {course['course_difficulty']}\n"
        f"교육 목표: {course['course_objective']}\n"
        f"수강 대상: {course['target_audience']}\n"
        f"수강 시간: {course['class_duration']}\n"
        f"수강 기간: {course['class_period']}\n"
        f"지원 기간: {course['support_period']}"
    ))

    process_and_chunk(offline_courses, lambda course: (
        f"강의 제목: {course['course_title']}\n"
        f"강의 분류: {course['course_category']}\n"
        f"교육 장소: {course['education_location']}\n"
        f"교육 목표: {course['educational_goals']}\n"
        f"수강 대상: {course['target_audience']}\n"
        f"지원 기간: {course['support_period']}\n"
        f"수강 기간: {course['class_period']}\n"
        f"지원 방법: {course['application_method']}\n"
        f"선발 절차: {course['selection_criteria']}\n"
        f"이수 기준: {course['completion_criteria']}"
    ))

    process_and_chunk(notice_info, lambda notice: (
        f"공지 제목: {notice['notice_title']}\n"
        f"공지 날짜: {notice['notice_date']}\n"
        f"공지 내용: {notice['notice_content']}"
    ))

    process_and_chunk(qna_info, lambda qna: (
        f"Q: {qna['question_title']}\n"
        f"질문: {qna['question_content']}\n"
        f"A: {qna['answer_content']}"
    ))

    process_and_chunk(review_info, lambda review: (
        f"강의 제목: {review['course_title']}\n"
        f"강의 별점: {review['rate']}\n"
        f"작성 날짜: {review['r_date']}\n"
        f"리뷰 내용: {review['content']}\n"
        f"리뷰 요약: {review['summarized_content']}"
    ))

    process_and_chunk(course_image_info, lambda image: (
        f"강의 제목: {image['course_title']}\n"
        f"강의 이미지 설명: {image['image_txt']}"
    ))

    return documents





# 텍스트를 일정한 길이로 청크 분할
def chunk_text(text, chunk_size=500, chunk_overlap=50):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    return text_splitter.split_text(text)

# 벡터 스토어 생성 (FAISS 사용)
def create_vector_store(documents):
    embeddings = OpenAIEmbeddings()
    vector_store = FAISS.from_documents(documents, embeddings)
    return vector_store

# 질문과 관련된 강의를 검색하고 답변을 생성하는 챗봇
def create_chatbot(vector_store, user_prompt, system_prompt):
    # OpenAI LLM 설정
    llm = OpenAI(temperature=0.2)

    # 프롬프트 템플릿 생성
    prompt_template = """
    시스템 프롬프트: {system_message}
    사용자 프롬프트: {user_message}
    사용자의 질문: {question}
    """

    prompt = PromptTemplate(
        input_variables=["system_message", "user_message", "question"],
        template=prompt_template
    )

    # PromptTemplate을 사용하여 LLM을 설정
    chat_model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.2)

    # ConversationalRetrievalChain 생성
    retrieval_chain = ConversationalRetrievalChain.from_llm(
        llm=chat_model,
        retriever=vector_store.as_retriever(),
        return_source_documents=True
    )

    return retrieval_chain

# 사용자 질문에 대한 답변을 생성하는 함수
def get_answer(question, retrieval_chain, chat_history):
    try:
        result = retrieval_chain({"question": question, "chat_history": chat_history})

        # 검색된 문서가 너무 많을 경우 일부만 사용하도록 설정
        source_documents = result.get('source_documents', [])
        max_docs = 5  # 최대 반환할 문서 개수 제한
        source_documents = source_documents[:max_docs]  # 최대 개수만큼 자르기

        answer = result.get('answer', '죄송합니다. 답변을 생성하는 중 오류가 발생했습니다. 🙏')

        return answer, source_documents
    except Exception as e:
        # 오류 발생 시 기본 안내 메시지 반환
        error_message = f"죄송합니다. 답변을 처리하는 중 오류가 발생했습니다. 😢\n오류 내용: {str(e)}"
        return error_message, []

# main 함수
def main():
    # 시스템 로케일을 'ko_KR.UTF-8'로 설정하여 한글 깨짐을 방지
    
    pickle_filename = "all_documents.pkl"
    # 1. 피클 파일이 있으면 로드하고, 없으면 데이터베이스에서 데이터를 가져와서 문서 생성
    all_documents = load_from_pickle(pickle_filename)

    
    locale.setlocale(locale.LC_ALL, 'ko_KR.UTF-8')
    if all_documents is None:
        # 강의 데이터 가져오기
        online_courses, offline_courses, notice_info, qna_info, review_info,course_image_info = fetch_courses()

        # 강의 정보를 문서로 변환
        documents = convert_to_documents(online_courses, offline_courses, notice_info, qna_info, review_info,course_image_info)

        all_documents = []
        all_documents.extend(documents)

        save_to_pickle(all_documents, pickle_filename)
    else:
        documents = all_documents

    # 벡터 스토어 생성
    vector_store = create_vector_store(documents)

    # 사용자 프롬프트와 시스템 프롬프트 정의
    system_prompt = """
    당신은 싹가능팀에서 만든 한국의 도시 서울에서 제공하는 취업 프로그램인 청년취업사관학교의 챗봇 '싹톡'입니다. 
    당신은 청년사관학교의 강의,공지사항에 관한 정보를 모두 갖고 있는 유능한 관리자입니다.
    대답할 때 반드시 적절한 이모지를 포함하세요! 🎉  
    예를 들면:  
    "오늘 날씨는 맑아요! ☀️"  
    "좋은 하루 보내세요! 😃"  

    """
    user_prompt = """
    당신에게 주어진 정보는 온라인/오프라인 강의 정보와 강의 리뷰, 질문답하기, 공지사항 등 청년사관학교와 관련된 데이터를 가지고 있으며, 그 외 질문에는 답변이 어렵다고 말하거나 정확한 질문을 요청하세요.
    당신은 질문에 대한 답변을 할때 다음의 원칙을 따릅니다.
    1. 오탈자 및 불필요한 기호 수정
    - 잘못된 문법, 의미 없는 특수문자, 깨진 문자(예: @, #, %) 등을 제거 혹은 수정합니다.
    2. 가독성 향상
    - 문장을 논리적으로 정리하여 가독성을 높이고, 관련된 내용을 묶어서 단락을 구성합니다.
    3. 목차 및 섹션 정리
    - 제목과 본문을 명확하게 구분합니다.
    4. 수업 관련 답변시 예시들기
    - 수업과 관련한 질문이 들어왔을시, 당신이 가지고 있는 정보를 최대한 활용한 예시를 같이 들어주세요.
    5. 다른 내용 참고하라고 말하지 않기.
    - 당신 외에 다른 담당자는 존재하지 않습니다. 

    """



    # 챗봇 생성
    retrieval_chain = create_chatbot(vector_store, user_prompt, system_prompt)

    # 대화 기록 초기화
    chat_history = []

    # Gradio 인터페이스 생성
    def chat_interface(user_input):
        try:
            # 답변 생성
            answer, _ = get_answer(user_input, retrieval_chain, chat_history)

            # 대화 기록에 (사용자 질문, 챗봇 답변) 추가
            chat_history.append((user_input, answer))

            # 챗봇의 응답을 (질문, 응답) 형태로 반환
            return [(user_input, answer)]
        except Exception as e:
            return [(user_input, f"더 명확하게 질문을 해주세요😃")]

    with gr.Blocks() as demo:
        gr.Markdown("""
        <div style="text-align: center;">
            <img src="https://sesac.seoul.kr/static/common/images/www/common/logo.png" alt="logo" style="display: inline; width: 25px;"/>
            <h1 style="display:inline; color: #6FC274;">SESAC</h1>
        </div>
        """)
        chatbot = gr.Chatbot()
        user_input = gr.Textbox(label="질문을 입력하세요", placeholder="질문을 입력하세요", interactive=True )
        user_input.submit(chat_interface, inputs=user_input, outputs=chatbot)

    demo.launch()

# 프로그램 실행
if __name__ == "__main__":
    main()


C:\Users\seul\AppData\Local\Temp\ipykernel_13508\2145101619.py:172: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()
C:\Users\seul\AppData\Local\Temp\ipykernel_13508\2145101619.py:179: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0.2)
C:\Users\seul\AppData\Local\Temp\ipykernel_13508\2145101619.py:194: LangChainDeprecationWarning: The 

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


C:\Users\seul\AppData\Local\Temp\ipykernel_13508\2145101619.py:208: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = retrieval_chain({"question": question, "chat_history": chat_history})
